## Question 1

Using all 5 stocks, assess if the efficient market hypothesis holds by making use of the random walk with a drift method.

## Question 2

Using the same data, calibrate the capital asset pricing (CAPM) model in each stock. Assess whether the efficient market hypothesis holds.

## Question 3

Compute the expected return of a portfolio, assuming these weights 
* 2 assets (Absa and Kumba)  (Weights 60%, 40%)
* 3 assets (Vodacom, Kumba and Woolworths) (Weights 30%, 30% and 40%)
* 3 assets (Absa, Vodacom Anglo American)   (Weights 25%, 25% and 50%)
* 4 assets (Anglo American, Kumba, Absa, and Vodacom) (Weights 15%, 30%, 35% and 20%)
* 5 assets (all 5 assets) (Weight 20%, 25%, 45%, 5% and 5%)

## Question 4

Compute the variance of a portfolio of
* 2 assets (Absa and Kumba)  (Weights 60%, 40%)
* 3 assets (Vodacom, Kumba and Woolworths) (Weights 30%, 30% and 40%)
* 3 assets (Absa, Vodacom Anglo American)   (Weights 25%, 25% and 50%)
* 4 assets (Anglo American, Kumba, Absa, and Vodacom) (Weights 15%, 30%, 35% and 20%)
* 5 assets (all 5 assets) (Weight 20%, 25%, 45%, 5% and 5%)


In [29]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm

In [30]:
# Read in data
df = pd.read_csv('Data/Finance_Exam_Data.csv')
df['Date'] = pd.to_datetime(df['Date'])
df['Risk- Free-Rate '] = df['Risk- Free-Rate '].fillna(0)
stocks = ['Vodacom','Anglo American','KUMBA','ABSA','Woolworths']


In [31]:
df.head()

,Date,Vodacom,Anglo American,KUMBA,ABSA,Woolworths,ALSE (benchmark),Risk- Free-Rate
0,2007-01-03,28.797146,2805.500000,11899.0,12700.0,23.501499,4.817180,0.185
1,2007-01-04,28.786953,2701.100098,11399.0,12600.0,23.214399,4.715587,0.190
2,2007-01-05,29.510704,2637.360107,11300.0,12286.0,23.006500,4.774849,0.180
3,2007-01-08,28.725790,2560.439941,11000.0,12370.0,23.214399,4.825645,0.175
4,2007-01-09,29.194698,2602.199951,11090.0,12170.0,22.580799,4.808713,0.000


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3353 entries, 0 to 3352
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Date              3353 non-null   datetime64[ns]
 1   Vodacom           3353 non-null   float64       
 2   Anglo American    3353 non-null   float64       
 3   KUMBA             3353 non-null   float64       
 4   ABSA              3353 non-null   float64       
 5   Woolworths        3353 non-null   float64       
 6   ALSE (benchmark)  3353 non-null   float64       
 7   Risk- Free-Rate   3353 non-null   float64       
dtypes: datetime64[ns](1), float64(7)
memory usage: 209.7 KB


## Question 1

In [33]:
# Function that takes in a timeseries (stock prices) and uses the Agumented Dickey-Fuller test to determine if the stock follows a random walk.
def emh_adf_test(timeseries):
    pval = adfuller(timeseries,regression='c')[1]
    if pval <= 0.05:
        print(f'With a p-value of {np.round(pval,5)}, we can reject the null hypothesis.\n{timeseries.name} does not follow a random walk\n')
    else:
        print(f'With a p-value of {np.round(pval,5)}, we fail to reject the null hypothesis.\n{timeseries.name} follows a random walk\n')

for stock in stocks:
    emh_adf_test(df[stock])

With a p-value of 0.50787, we fail to reject the null hypothesis.
Vodacom follows a random walk

With a p-value of 0.37853, we fail to reject the null hypothesis.
Anglo American follows a random walk

With a p-value of 0.37645, we fail to reject the null hypothesis.
KUMBA follows a random walk

With a p-value of 0.08644, we fail to reject the null hypothesis.
ABSA follows a random walk

With a p-value of 0.2675, we fail to reject the null hypothesis.
Woolworths follows a random walk



In [34]:
# Calculates the return for each stock, and adds those returns to the dataframe.
for stock in stocks:
    df[f'{stock} Return'] = df[stock].pct_change(periods=1)
    
# Calculates the excess market return rate
df['Excess Market Return'] = df['ALSE (benchmark)'] - df['Risk- Free-Rate ']
df['Excess Market Return'].iloc[0] = np.nan

# Calculates the excess return for each stock
for stock in stocks:
    df[f'Excess {stock}'] = df[f'{stock} Return'] - df['Risk- Free-Rate ']

C:\Users\clint\AppData\Local\Temp\ipykernel_16480\2302573555.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Excess Market Return'].iloc[0] = np.nan


## Question 2

In [53]:
# Function that takes in a dataframe, and list of stock names, and uses the capital asset pricing model to determine each stock's alpha.
# The alpha will give info on whether abnormal profits are possible.

def capm_alpha_significance(dataset,stock):
    # Create independent and dependent variables for the model
    y = dataset[f'Excess {stock}']
    X = dataset[f'Excess Market Return']
    X = sm.add_constant(X)

    # Create the model
    model = sm.OLS(y,X,missing='drop')
    results = model.fit()

    # Store the value of alpha and it's p-value 
    const_pvalue = results.pvalues['const']
    const_coeff = results.params['const']

    # Checks if alpha is significant and if it is negative
    if const_pvalue <= 0.05 and const_coeff < 0:
        print(f'Alpha is significant (p-value: {np.round(const_pvalue,4)}), however it is negative (alpha: {np.round(const_coeff,4)}), therefore abnormal profits are not possible. EMH for {stock} holds.\n')
    
    # Checks if alpha is significant and if it is positive
    elif const_pvalue <= 0.05 and const_coeff > 0:
        print(f'Alpha is positive (alpha: {np.round(const_coeff,4)}) and signifianct (p-value: {np.round(const_pvalue,4)}) , therefore abnormal profits are possible. EMH for {stock} does not hold.\n')
    
    # If alpha is insignificant or if it is 0
    else:
        print(f'Alpha is insignificant (p-value: {np.round(const_pvalue,4)}). EMH for {stock} holds.\n')
    pass

# Loop through each stock to determine the alpha
for stock in stocks:
    capm_alpha_significance(df,stock=stock)

Alpha is significant (p-value: 0.0), however it is negative (alpha: -1.5497), therefore abnormal profits are not possible. EMH for Vodacom holds.

Alpha is significant (p-value: 0.0), however it is negative (alpha: -1.4973), therefore abnormal profits are not possible. EMH for Anglo American holds.

Alpha is significant (p-value: 0.0), however it is negative (alpha: -1.546), therefore abnormal profits are not possible. EMH for KUMBA holds.

Alpha is significant (p-value: 0.0), however it is negative (alpha: -1.5492), therefore abnormal profits are not possible. EMH for ABSA holds.

Alpha is significant (p-value: 0.0), however it is negative (alpha: -1.5477), therefore abnormal profits are not possible. EMH for Woolworths holds.



In [36]:
# Take the mean return for each stock and store it in a dictionary
expected_returns = {
    'Vodacom':df['Vodacom Return'].mean(),
    'Anglo American':df['Anglo American Return'].mean(),
    'KUMBA':df['KUMBA Return'].mean(),
    'ABSA':df['ABSA Return'].mean(),
    'Woolworths':df['Woolworths Return'].mean()
}

# Convert the dictionary to a pandas dataframe
expected_returns_df = pd.DataFrame(expected_returns,index=['Expected Returns'])

expected_returns_df

,Vodacom,Anglo American,KUMBA,ABSA,Woolworths
Expected Returns,0.000067,0.023503,0.000797,0.000205,0.00023


## Question 3

In [37]:
# Function that takes in the expected returns of each stock, and the weights for each stock, in order to calculate that portfolio's expected return
def expected_portfolio_return(expected_returns_stocks_df, weights):
    # Calculates the sum of the products of a stock's expected return and it's weight in the portfolio
    ret = np.dot(expected_returns_stocks_df,weights)*100
    print(f'The given stocks and weights has a return of {np.round(ret,4)} percent.')


In [38]:
expected_portfolio_return(expected_returns_df[['ABSA','KUMBA']],[0.6,0.4])

The given stocks and weights has a return of [0.0442] percent.


In [39]:
expected_portfolio_return(expected_returns_df[['Vodacom','KUMBA','Woolworths']],[0.3,0.3,0.4])

The given stocks and weights has a return of [0.0351] percent.


In [40]:
expected_portfolio_return(expected_returns_df[['ABSA','Vodacom','Anglo American']],[0.25,0.25,0.5])

The given stocks and weights has a return of [1.182] percent.


In [41]:
expected_portfolio_return(expected_returns_df[['Anglo American','KUMBA','ABSA','Vodacom']],[0.15,0.30,0.35,0.20])

The given stocks and weights has a return of [0.385] percent.


In [42]:
expected_portfolio_return(expected_returns_df,[0.2,0.25,0.45,0.05,0.05])

The given stocks and weights has a return of [0.6269] percent.


## Question 4

In [43]:
# Creates an excess retun matrix by subtracting the expected return for the stock from the current return.
excess_return_matrix = pd.DataFrame()
for stock in stocks:
    excess_return_matrix[f'{stock} Excess'] = df[f'{stock} Return'] - expected_returns_df[stock][0]

# Creates a variance covariance matrix
variance_covariance_matrix = excess_return_matrix.cov()

In [44]:
# The symetrical variance covariance matrix displays the variance of the excess returns on the diagonal, and the covariance between the stocks in all the other positions.
variance_covariance_matrix

,Vodacom Excess,Anglo American Excess,KUMBA Excess,ABSA Excess,Woolworths Excess
Vodacom Excess,0.000362,-0.000187,0.000024,0.000014,0.000009
Anglo American Excess,-0.000187,1.826478,0.000831,0.000668,0.000043
KUMBA Excess,0.000024,0.000831,0.000943,0.000133,0.000016
ABSA Excess,0.000014,0.000668,0.000133,0.000366,0.000003
Woolworths Excess,0.000009,0.000043,0.000016,0.000003,0.000183


In [45]:
# Function that takes in asset names, and asset weights, in order to calculate the variance of the portfolio
def portfolio_variance(asset_names,asset_weights):
    asset_weights = np.array(asset_weights)
    var_mat = variance_covariance_matrix.loc[asset_names,asset_names].to_numpy()
    # Nested matrix multiplicatoin of the (asset weights * variance covariance matrix) and the asset weights
    var = np.matmul(np.matmul(asset_weights,var_mat),asset_weights.T)
    # Prints the variance for the portfolio
    print(f'Variance: {var}')

In [46]:
portfolio_variance(asset_names=['ABSA Excess','KUMBA Excess'],asset_weights=[0.6,0.4])

Variance: 0.0003462876162892855


In [47]:
portfolio_variance(asset_names=['Vodacom Excess','KUMBA Excess','Woolworths Excess'],asset_weights=[0.3,0.3,0.4])

Variance: 0.00015712833598748152


In [48]:
portfolio_variance(asset_names=['ABSA Excess','Vodacom Excess','Anglo American Excess'],asset_weights=[0.25,0.25,0.5])

Variance: 0.4567868723082937


In [49]:
portfolio_variance(asset_names=['Anglo American Excess','KUMBA Excess','ABSA Excess','Vodacom Excess'],asset_weights=[0.15,0.3,0.35,0.2])

Variance: 0.04140633865857087


In [50]:
portfolio_variance(asset_names=variance_covariance_matrix.columns,asset_weights=[0.2,0.25,0.45,0.05,0.05])

Variance: 0.1145592003878631
